Install azure cli, login with 'az login' get the PROJECT_CONNECTION_STRING with 'az ml workspace show -n {project_name} --resource-group {resource_group_name} --query discovery_url' and set it as an env variable

In [3]:
!pip install azure-ai-projects azure-identity --quiet

In [3]:
import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather,
}

In [1]:
# Read env variables from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [8]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
# from user_functions import user_functions # user functions which can be found in a user_functions.py file.

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model="gpt-4o-mini", name="my-agent", instructions="You are a weather bot. Use the provided functions to help answer questions.", toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_38l0D0Bux7KV1EgCoLtSoUUp


In [9]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Hello, send an email with the datetime and weather information in New York?",
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_64t37lwaumCO6ikyoK843zJE
Created message, ID: msg_iSoPOyZua1wvBI64R8cJhbot


In [10]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the agent when done
# project_client.agents.delete_agent(agent.id)
# print("Deleted agent")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: failed
Run failed: {'code': 'rate_limit_exceeded', 'message': 'Rate limit is exceeded. Try again in 60 seconds.'}
Messages: {'object': 'list', 'data': [{'id': 'msg_iSoPOyZua1wvBI64R8cJhbot', 'object': 'thread.message', 'created_at': 1742662393, 'assistant_id': None, 'thread_id': 'thread_64t37lwaumCO6ikyoK843zJE', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, send an email with the datetime and weather information in New York?', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_iSoPOyZua1wvBI64R8cJhbot', 'last_id': 'msg_iSoPOyZua1wvBI64R8cJhbot', 'has_more': False}
